In [ ]:
!pip install konlpy
!pip install lexrankr

     |████████████████████████████████| 19.4 MB 207 kB/s 
     |████████████████████████████████| 448 kB 27.1 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
import random

In [ ]:
#정제한 자기소개서 질문 파일('final.txt') 불러오기
data = pd.read_csv('final.txt', sep='\n', names=['면접질문'], low_memory=False)
data.head(2)

,면접질문
0,면접질문
1,"1, 2, 3차 산업혁명에 대해 말해보세요."


In [ ]:
data['면접질문'].isnull().sum() #null 값이 들어있는 행의 개수
data['면접질문'] = data['면접질문'].fillna('') #0행에 null이 들어있다면 ''로 대체

In [ ]:
# 특정 단어 추출(by. okt) & 유사도 측정(by. tf-idf)
okt = Okt()
# tokenizer : 문장에서 색인어 추출을 위해 명사,동사,알파벳,숫자 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
def oktTokenizer(raw, pos=["Noun","Alpha"], stopword=['로서', '면', '일지', '위해', '대해', '무엇', '어디', '또한', '대한', '통해', '분야', '생각', '때문', '업무']): #,"Verb","Number"
    return [
        word for word, tag in okt.pos(
            raw,
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

In [ ]:
tfidf = TfidfVectorizer(tokenizer=oktTokenizer, ngram_range=(2,3))

In [ ]:
# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['면접질문'])

(5690, 20995)


In [ ]:
# 문장에서 뽑아낸 feature 들의 배열
features = tfidf.get_feature_names()

['ai 빅데이터', 'ai 빅데이터 차이', 'ai 의견', 'ai 이해', 'ai 이해 불가능', 'ai 채용', 'ai 채용 도입', 'ai 학습', 'ai 학습 챗봇', 'ai 회사', 'ai 회사 근무', 'algorithm 하나', 'algorithm 하나 설명', 'alive 설명', 'aop ioc', 'aop ioc 설명', 'ar 게임', 'ar 게임 과연', 'attack 설명', 'back 개발', 'backward 방식', 'backward 방식 forward', 'bigdata mobile', 'bigdata mobile 개념', 'bread work', 'bread work for', 'business school', 'career goal', 'cash 암호', 'cash 암호 알고리즘', 'catia 프로그램', 'cctv 설치', 'cctv 설치 교사', 'cctv 설치 본인', 'ceo 강압', 'ceo 강압 비도덕적', 'ceo 라면', 'ceo 라면 매뉴얼', 'ceo 이유', 'ceo 청렴', 'ceo 청렴 실적', 'char 차이', 'city 영어', 'city 영어 설명', 'cj 나영석', 'cj 나영석 pd', 'cloud bigdata', 'cloud bigdata mobile', 'cloud 회사', 'cloud 회사 방식', 'compareto 소드', 'compareto 소드 설명', 'controller switch', 'controller switch 간의', 'creative 이유', 'cs 기본', 'cs 어떡', 'cs 어떡 직무', 'cs 엔지니어', 'cs 엔지니어 설비', 'cs 엔지니어 출장', 'cs 프로그래밍', 'cs 프로그래밍 인트라넷', 'db procedure', 'db 설계자', 'db 설계자 프로그래머', 'db 속도', 'db 속도 증가', 'db 인덱싱', 'db 인서트', 'db 자격증', 'db 자격증 가지', 'db 정규', 'db 정규 화란', 'dna 설명', '

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# tokenizer : 문장에서 색인어 추출을 위해 명사,동사,알파벳,숫자 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
def oktNtokenizer(raw, stopword=['로서', '면', '일지', '위해', '대해', '무엇', '어디', '또한', '대한', '통해', '분야', '생각', '때문', '업무']):
    pos=["Noun","Alpha"]
    
    tokenlist = okt.pos(
        raw,
        norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
        stem=True    # stemming 바뀌나->바뀌다
        )
    
    #단어 추출
    wordlist = []
    for t in tokenlist:
      #print(t)
      word = t[0]
      tag = t[1]
      if len(word) > 1 and tag in pos and word not in stopword:
        wordlist.append(word)
    #print(wordlist)

    #2개 단어씩 묶음
    bow2 = []
    for i in range(len(wordlist)-1):
      for j in range(len(wordlist)-1):
        if wordlist[i]!=wordlist[j]:
          bows = wordlist[i]+' '+wordlist[j]
          bow2.append(bows)
    #print(bow2)
    
    #3개 단어씩 묶음
    bow3 = []
    for i in range(len(wordlist)-1):
      for j in range(len(wordlist)-1):
        for k in range(len(wordlist)-1):
          if wordlist[i]!=wordlist[j] and wordlist[j]!=wordlist[k] and wordlist[k]!=wordlist[i]:
            bows = wordlist[i]+' '+wordlist[j]+' '+wordlist[k]
            bow3.append(bows)
    #print(bow3)

    bowtoken = wordlist + bow2 + bow3
    bowtoken = list(set(bowtoken))  #중복 제거를 위해 set으로 변환했다가 다시 list로
    return bowtoken

In [ ]:
def get_recommendations(title):
    # 검색 문장에서 feature를 뽑아냄
    srch=[t for t in oktNtokenizer(title) if t in features]
    #print(srch)

    # dtm 에서 검색하고자 하는 feature만 뽑아낸다.
    srch_dtm = np.asarray(tfidf_matrix.toarray())[:, [
    # tfidf.vocabulary_.get 는 특정 feature 가 dtm 에서 가지고 있는 index값을 리턴한다
    tfidf.vocabulary_.get(i) for i in srch]]

    score = srch_dtm.sum(axis=1)

    similar = {} #dictiondary 선언('면접질문'&'유사도 측정치'를 받음)
    for i in score.argsort()[::-1]:
        #if score[i] > 0:
            #print('{} / score : {}'.format(data['면접질문'][i], score[i]))
            similar[data['면접질문'][i]] = score[i]

    return similar

In [ ]:
# 자소서 데이터로 유사도 측정하기
# 문장 단위로 유사한 면접질문 뽑기

def sentsimilar(resume):
  global sim
  global nos
  f_result = []
  sentence = resume.split('.')
  for i in range(len(sentence)-1):
    sentence[i].replace("\n", '')
    result = get_recommendations(sentence[i])
    if result == [] :
      pass
    else :
      sim = []
      nos = []
      l = list(zip(result.keys(), result.values()))
      for j in range(len(l)):
        if (l[j][1]) > 0.35:
          temp = []
          temp.append(sentence[i])
          temp.append(l[j][0])
          temp.append(1)
          sim.append(temp) 
        if (l[j][1]) == 0.0:
          temp = []
          temp.append(sentence[i])
          temp.append(l[j][0])
          temp.append(0)
          nos.append(temp)
      if len(nos) > len(sim) :
        new_n = random.sample(nos, len(sim))
        for k in range(len(sim)):
          f_result.append(sim[k])
          f_result.append(new_n[k])
      else :
        for k in range(len(nos)):
          f_result.append(sim[k])
          f_result.append(nos[k])
       
  return f_result

In [ ]:
#실행
#lexrankr 패키지를 사용해 문단별로 요약해둔 자기소개서 데이터(자소서문단.txt) 불러와서 실행

In [ ]:
f = open("자소서문단.txt", 'r')
resumeList = f.readlines()
for line in resumeList:
    line = line.strip() 

4107
<class 'list'>


In [ ]:
final = []
for i in resumeList :
  temp = sentsimilar(i)
  final += temp

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
df = pd.DataFrame(final, columns=['자소서문장', '면접질문', '일치여부'])

df.to_excel('dataset.xlsx', index=False)